In [ ]:
from os import path
import sys
sys.path.append(path.abspath('../'))

from collections import defaultdict
from pathlib import Path
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split

from src.process_data import DefaultSmilesFeaturizer

In [2]:
#!python3 -m pip install tensorflow

In [3]:
DATA_PATH = Path("../resources/data/polyinfo.xlsx")

df = pd.read_excel(DATA_PATH.as_posix())

## Transform data

In [4]:
all_properties = set(df.property_name)
new_df = defaultdict(lambda: {key: None for key in all_properties})

for index, row in df.iterrows():
    new_df[row["polymer_id"]][row["property_name"]] = row["property_value_median"]
    new_df[row["polymer_id"]]["polymer_id"] = row["polymer_id"]

new_df = pd.DataFrame(list(new_df.values()))


In [ ]:
new_df

## Apply smiles featurizer

In [6]:
new_df = new_df.dropna(subset=['SMILES'])
len(new_df)

18641

In [ ]:
featurizer = DefaultSmilesFeaturizer()

smiles_df = new_df["SMILES"].apply(featurizer).apply(pd.Series)

In [8]:
new_df = new_df[~smiles_df[0].isna()]
len(new_df)

18618

In [9]:
smiles_df = smiles_df[~smiles_df[0].isna()]
len(smiles_df)

18618

In [10]:
new_df = new_df.drop(columns=["SMILES", 'polymer_id'])
len(new_df)

18618

In [11]:
TARGET_COLUMNS = [k
                  for k, v in dict(new_df.count()).items()
                  if v > 1000]
TARGET_COLUMNS.remove("Electric conductivity")

In [12]:
import matplotlib.pyplot as plt

In [ ]:
target_df = new_df[TARGET_COLUMNS].copy()
dfs = {}
for column in TARGET_COLUMNS:
    target = new_df[column].copy()
    index = ~target.isna() & (target < target.quantile(.95)) & (target > target.quantile(.05))
    dfs[column] = (target[index], smiles_df[index])

print({k: len(v[0]) for k, v in dfs.items()})

#### Data dist

In [ ]:
grid_x = 3
fig, axs = plt.subplots(int(np.ceil(len(TARGET_COLUMNS) / grid_x)), grid_x, figsize=(12, 12))
for i, target in enumerate(TARGET_COLUMNS):
    ax = axs[int(i / grid_x), i % grid_x]
    ax.set_title(target)
    ax.hist(dfs[target][0], bins=50)

## Model

In [15]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K

@keras.saving.register_keras_serializable()
class QuantileLoss(keras.losses.Loss):
    def __init__(self, perc, huber_delta=1e-4, size_alpha=0.):
        super().__init__(reduction=keras.losses.Reduction.AUTO, 
                         name="quantile_loss")
        if not isinstance(perc, list):
            perc = [perc]
        assert all([0 < _q < 1 for _q in perc])
        self.__perc = perc

        perc = np.array(perc).reshape(-1)
        perc.sort()
        self._perc = K.constant(perc.reshape(1, -1))

        assert 0 < huber_delta < 1
        self._delta = huber_delta
        self._size_alpha = size_alpha

    def get_config(self):
        return {
            "perc": self.__perc,
            "huber_delta": self._delta,
            "size_alpha": self._size_alpha
        }

    def _huber_quantile_loss(self, y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - self._perc) + (1 - I) * self._perc
        huber_loss = K.sum(correction * tf.where(d <= self._delta, 
                                                 0.5 * d ** 2 / self._delta, 
                                                 d - 0.5 * self._delta), -1)
        return huber_loss

    def call(self, y, pred):
        huber_loss = self._huber_quantile_loss(y, pred)
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        q_int_size_loss = K.sum(tf.abs(pred[:, 2] - pred[:, 0])) * self._size_alpha
        return huber_loss + q_order_loss + q_int_size_loss
    
@keras.saving.register_keras_serializable()
class IntervalAccuracy(tf.keras.metrics.Accuracy):
    def update_state(self, y_true, y_pred, sample_weight=None):
        matches = tf.logical_and(y_pred[:, 0] < y_true, y_true < y_pred[:, 2])
        return super().update_state(tf.ones_like(matches), matches, 
                                    sample_weight=sample_weight)

@keras.saving.register_keras_serializable()
class RelativeIntervalSize:
    def __init__(self, mean_value):
        self.mean_value = mean_value

    def get_config(self):
        return {"mean_value": self.mean_value}
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

    def __call__(self, y_true, y_pred):
        return K.mean(tf.abs(y_pred[:, 2] - y_pred[:, 0])) / self.mean_value

@keras.saving.register_keras_serializable()
def q_mae(y_true, y_pred):
    return K.mean(tf.abs(y_pred[:, 1] - y_true))

@keras.saving.register_keras_serializable()
def q_mape(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_percentage_error(
        y_true, y_pred[:, 1]
    ) / 100

@keras.saving.register_keras_serializable()
def mape(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred) / 100

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error, accuracy_score, mean_absolute_error, r2_score

def plot_loss(history, key_to_label, ymax=0.5):
    for k, v in key_to_label.items():
        plt.plot(history.history[k], label=v)
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.ylim(0, ymax)
    plt.grid(True)

def plot_target_pred(model, X_test, y_test, ax=None, with_interval=False, verbose=True):
    metrics = {}
    prediction = []
    index = 1 if with_interval else 0

    prediction = model.predict(X_test, verbose=0)
    
    target = y_test.to_numpy()
    ind = np.argsort(prediction[:, index])
    target = target[ind]
    prediction = prediction[ind]

    mae = mean_absolute_error(target, prediction[:, index]) / np.mean(np.abs(target))
    metrics["FMAE"] = mae
    print("FMAE: {:.2f}".format(mae))
    r2 = r2_score(target, prediction[:, index])
    metrics["R2"] = r2
    print("R2: {:.2f}".format(r2))

    colors = ['g'] * len(target)
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    if with_interval:
        matches = np.logical_and(prediction[:, 0] < target,
                                 target < prediction[:, 2])
        accuracy = np.mean(matches.astype(np.float32))
        relative_size = np.mean(np.abs(prediction[:, 0] - prediction[:, 2])) / (np.max(target) - np.min(target))
        metrics["accuracy"] = accuracy
        metrics["mean_relative_interval_size"] = relative_size
        colors = ['g' if x else 'r' for x in matches]

        print("Mean relative interval size: {:.2f}".format(relative_size))
        print("Accuracy: {:.2f}".format(accuracy))
        ax.fill_between(x=prediction[:, 1], 
                        y1=prediction[:, 0], y2=prediction[:, 2], alpha=0.3, color='g')
    
    ax.set_xlim(np.min(prediction[:, index]), np.max(prediction[:, index]))
    ax.set_ylim(np.min(target), np.max(target))
    ax.plot(np.array([np.min(target), np.max(target)]), np.array([np.min(target), np.max(target)]), color='r')
    ax.scatter(prediction[:, index], target, color=colors, s=1)

    return metrics



In [37]:
QUANTILES = [0.05, 0.5, 0.95]

In [38]:
def make_model_for_target(ax, target):
    data = pd.concat(dfs[target], axis=1)

    data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

    X_train = data_train[smiles_df.columns]
    y_train = data_train[target]
    X_test = data_test[smiles_df.columns]
    y_test = data_test[target]

    print(f"Training model for target {target}.\nData size {len(X_train)} train, {len(X_test)} test")

    quantile_loss = QuantileLoss(QUANTILES, size_alpha=5e-4)
    model = keras.Sequential([
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(len(QUANTILES)),
    ])

    mean_interval_size = RelativeIntervalSize(np.mean(y_train))
    # IF YOU HAVE M1/M2 MAC
    model.compile(loss=[quantile_loss],
                  metrics=[q_mape, mean_interval_size, IntervalAccuracy()],
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    model.fit(X_train, y_train, verbose=0, validation_split=0.2, epochs=300)
    
    metrics = plot_target_pred(model, X_test, y_test, ax=ax, with_interval=True, verbose=False)
    return model, metrics

### Try one target: Density

In [39]:
target = "Density"
data = pd.concat(dfs[target], axis=1)

In [40]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

X_train = data_train[smiles_df.columns]
y_train = data_train[target]
X_test = data_test[smiles_df.columns]
y_test = data_test[target]

In [41]:
len(X_train), len(X_test)

(1260, 316)

### MSE exp

In [ ]:
model = keras.Sequential([
      layers.Dense(16, activation='relu'),
      layers.Dense(8, activation='relu'),
      layers.Dense(1),
  ])

# IF YOU HAVE M1/M2 MAC
model.compile(loss=[keras.losses.mse],
              metrics=[mape, keras.metrics.mae],
              optimizer=tf.keras.optimizers.Adam(0.001))

In [43]:
history = model.fit(X_train, y_train, verbose=0, validation_split=0.2, epochs=300)

In [ ]:
plot_loss(history,
          {"val_mape": "val MAPE", "val_mean_absolute_error": "val MAE"})

In [ ]:
plot_target_pred(model, X_test, y_test)

### Quantiles

In [ ]:
quantile_loss = QuantileLoss(QUANTILES, size_alpha=5e-4)
q_model = keras.Sequential([
      layers.Dense(16, activation='relu'),
      layers.Dense(8, activation='relu'),
      layers.Dense(len(QUANTILES)),
  ])

mean_interval_size = RelativeIntervalSize(np.mean(y_train))
q_model.compile(loss=[quantile_loss],
                metrics=[q_mape, q_mae, mean_interval_size, IntervalAccuracy()],
                optimizer=tf.keras.optimizers.Adam(0.001))

# OTHERWISE
# model.compile(loss='mean_squared_error',
#               optimizer=tf.keras.optimizers.Adam(0.001))

In [47]:
history = q_model.fit(X_train, y_train, verbose=0, validation_split=0.2, epochs=200)

In [ ]:
plot_loss(history,
          {"val_relative_interval_size": "mean relative interval size",
           "val_accuracy": "val accuracy",
           "val_q_mae": "val MAE",
           "val_q_mape": "val MAPE"},
           ymax=1.)

In [ ]:
plot_target_pred(q_model, X_test, y_test, with_interval=True)

### All targets

In [ ]:
metrics_per_target = {}
grid_x = 4
fig, axs = plt.subplots(int(np.ceil(len(TARGET_COLUMNS) / grid_x)), grid_x, figsize=(12, 12))
for i, target in enumerate(TARGET_COLUMNS):
    ax = axs[int(i / grid_x), i % grid_x]
    ax.set_title(target)
    metrics_per_target[target] = make_model_for_target(ax, target)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame({target: {k: round(v, 2) for k, v in metrics.items()} 
              for target, (_, metrics) in metrics_per_target.items()})

## Make table prediction for SMILES

In [52]:
def make_prediction(smiles_string):
    smiles_features = np.array([featurizer(smiles_string)])
    prediction = []
    for i, target in enumerate(TARGET_COLUMNS):
        model, metrics = metrics_per_target[target]
        model_prediction = model(smiles_features)[0].numpy()
        prediction.append({
            "Target": target,
            "Predicted median": round(model_prediction[1], 2),
            "Interval": (round(model_prediction[0], 2), round(model_prediction[2], 2)),
            "Interval accuracy": round(metrics["accuracy"], 2)
        })
    return pd.DataFrame(prediction)

In [53]:
pd.options.display.float_format = '{:,.2f}'.format
make_prediction("*CCC1CC(C2C1CCC2)*").to_csv("prediction.csv")

### Save model

In [54]:
from src.model import NNModelWrapper
        

In [55]:
NNModelWrapper.save_model(Path("../resources/models/nn"), metrics_per_target)

In [ ]:
mm = NNModelWrapper(Path("../resources/models/nn"))
# Model interface is exactly the same as it was with XGB, see src/model.py
# output is {target: [lower bound, median, upper bound]}
# interesting metric is accuracy (accuracy of the real value appearance inside interval)

# example: [0.5, 0.57, 0.7], 0.8
# value is in [0.5, 0.7], error probability 20%
mm("CCC")